In [1]:
from pathlib import Path
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torchvision import transforms

from dataloading.nvidia import NvidiaResizeAndCrop, Normalize, NvidiaDataset
from network import PilotNet

%load_ext autoreload
%autoreload 2

In [2]:
root_path = Path("/home/romet/data/datasets/ut/nvidia-data")
valid_paths = [root_path / "2021-05-20-13-59-00_e2e_sulaoja_10_10"]
#tr = transforms.Compose([NvidiaResizeAndCrop(), Normalize()])
validset = NvidiaDataset(valid_paths)

In [3]:
def load_model(model_path):
    model = PilotNet()
    model.load_state_dict(torch.load(model_path))
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    return model

In [4]:
model = load_model("models/1-pilotnet-base/1cam/best.pt")

In [5]:
def predict(model, dataset):
    loader = DataLoader(dataset, batch_size=len(dataset),  # TODO: batching
                        shuffle=False, num_workers=0, pin_memory=False, persistent_workers=False)
    dataiter = iter(loader)

    predictions = []
    true_steering_angles = []

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        data = dataiter.next()
        inputs = data['image'].to(device)
        predictions.extend(model(inputs).cpu().flatten())
        true_steering_angles.extend(data['steering_angle'].to(device))

    return data['steering_angle'], predictions

In [6]:
true_steering_angles, predicted_steering_angles = predict(model, validset)

plt.figure(figsize=(7, 7))
plt.plot(true_steering_angles)
plt.plot(predicted_steering_angles);
plt.legend(["ground truth", "prediction"])

RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [ ]:
loader = DataLoader(validset, batch_size=5, shuffle=False, num_workers=0)
dataiter = iter(loader)
data = dataiter.next()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inputs = data['image'].to(device)
print(inputs.shape)

In [ ]:
ONNX_FILE_PATH = 'models/1-pilotnet-base/1cam/best.onnx'
torch.onnx.export(model, inputs, ONNX_FILE_PATH)


In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
import tensorrt as trt

# logger to capture errors, warnings, and other information during the build and inference phases
TRT_LOGGER = trt.Logger()

def build_engine(onnx_file_path):
    # initialize TensorRT engine and parse ONNX model
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network()
    parser = trt.OnnxParser(network, TRT_LOGGER)
    
    # parse ONNX
    with open(onnx_file_path, 'rb') as model:
        print('Beginning ONNX file parsing')
        parser.parse(model.read())
    print('Completed parsing of ONNX file')